In [52]:
import pandas as pd
import random
from collections import Counter 
import operator

In [16]:
data_file = 'capture20110815-2.pcap.netflow.labeled' # dataset 43
data_file = 'capture20110811.pcap.netflow.labeled' # dataset 46
data_file = 'capture20110816-3.pcap.netflow.labeled	' # dataset 49

columns=['Date','Duration','Protocol','SrcIP', 'srcPort','DstIP', 'DstPort', 'Flags', 'TOS', 'Packets', 'Bytes', 'Label']
lst=[]
with open(data_file) as fp:
    for cnt, line in enumerate(fp):
        if cnt!=0:
            data=line.split("\t")
            data = list(filter(None, data)) 
            date = data[0]
            duration = data[1]
            protocol = data[2]
            src = data[3].split(':')
            if len(src) < 2: # Set port to 0 if no port provided
                src.append(0)
            dst = data[5].split(':')
            if len(dst) < 2:
                dst.append(0)
            flags = data[6]
            tos = data[7]
            packets = data[8]
            byteno = data[9]
            flows = data[10]
            label = data[11].rstrip()
            lst.append([date, duration, protocol, src[0], src[1], dst[0], dst[1], flags, tos, packets, byteno, label])
df=pd.DataFrame(lst, columns=columns)
df.head()

,Date,Duration,Protocol,SrcIP,srcPort,DstIP,DstPort,Flags,TOS,Packets,Bytes,Label
0,2011-08-11 10:10:00.003,0.967,UDP,89.31.8.11,23929,147.32.84.229,13363,INT,0,2,135,Background
1,2011-08-11 10:10:00.003,0.967,UDP,147.32.84.229,13363,89.31.8.11,23929,INT,0,2,276,Background
2,2011-08-11 10:10:00.006,0.000,UDP,208.88.186.6,34042,147.32.84.229,13363,INT,0,1,62,Background
3,2011-08-11 10:10:00.008,0.000,UDP,92.118.218.77,55246,147.32.84.229,13363,INT,0,1,78,Background
4,2011-08-11 10:10:00.009,0.000,UDP,182.185.139.181,10223,147.32.84.229,13363,INT,0,1,72,Background


In [17]:
infected_host = '147.32.84.165'
df_infected=df.loc[(df['SrcIP']==infected_host) | (df['DstIP']==infected_host)]
df_infected=df_infected.reset_index()

In [18]:
# Count connections
ip_map = {}
for index, row in df_infected.iterrows():
    if row["SrcIP"] == infected_host:
        ip_map.setdefault(row["DstIP"],0)
        ip_map[row["DstIP"]] += 1
    else:
        ip_map.setdefault(row["SrcIP"],0)
        ip_map[row["SrcIP"]] += 1

In [65]:
# Show the 10 most frequestly connecting IPs 
print(Counter(ip_map).most_common(10))
assert len(df_infected) == sum(Counter(ip_map).values())
len(df_infected)

[('193.23.181.44', 6442), ('174.128.246.102', 4101), ('174.37.196.55', 3707), ('173.236.31.226', 3410), ('184.154.89.154', 3344), ('67.19.72.206', 3224), ('46.4.36.120', 3150), ('72.20.15.61', 3111), ('147.32.80.9', 1573), ('212.117.171.138', 967)]


54433

In [35]:
# Calculate the exact fequencies
for ip in Counter(ip_map).most_common(10):
    print(f"{ip[0]} : {(ip[1] / len(df_infected) * 100)}%")


193.23.181.44 : 11.834732607058218%
174.128.246.102 : 7.534032664008965%
174.37.196.55 : 6.810207043521394%
173.236.31.226 : 6.264582146859442%
184.154.89.154 : 6.143332169823452%
67.19.72.206 : 5.9228776661216544%
46.4.36.120 : 5.786930722172212%
72.20.15.61 : 5.715283008469127%
147.32.80.9 : 2.8897911193577426%
212.117.171.138 : 1.7764958756636597%


In [58]:
# A function to randomly select 
# k items from stream[0..n-1]. 
def selectKItems(stream, n, k): 
    i=0
    # index for elements 
    # in stream[] 

    # reservoir[] is the output  
    # array. Initialize it with 
    # first k elements from stream[] 
    reservoir = [0]*k
    for i in range(k): 
        if (df_infected.iloc[0, :]["DstIP"] == infected_host):
            reservoir[i] = df_infected.iloc[i, :]["SrcIP"]
        else:
            reservoir[i] = df_infected.iloc[i, :]["DstIP"]
            
    # Iterate from the (k+1)th 
    # element to nth element 
    while(i < n): 
        # Pick a random index 
        # from 0 to i. 
        j = random.randrange(i+1)

        # If the randomly picked 
        # index is smaller than k, 
        # then replace the element 
        # present at the index 
        # with new element from stream 
        if(j < k): 
            if (df_infected.iloc[i, :]["DstIP"] == infected_host):
                reservoir[j] = df_infected.iloc[i, :]["SrcIP"]
            else:
                reservoir[j] = df_infected.iloc[i, :]["DstIP"]
        i+=1

    return reservoir

In [63]:
reservoirSizes = [100, 1000, 5000, 10000]
for reservoir in reservoirSizes:
    print(f"\n\nReservoir size = {reservoir}")
    kfreqIPS = selectKItems(df_infected, len(df_infected), reservoir)
    for ip in Counter(kfreqIPS).most_common(10):
        print(f"{ip[0]} : {(ip[1] / len(kfreqIPS) * 100)}%")



Reservoir size = 100
193.23.181.44 : 17.0%
72.20.15.61 : 8.0%
67.19.72.206 : 7.000000000000001%
174.37.196.55 : 6.0%
184.154.89.154 : 5.0%
173.236.31.226 : 5.0%
174.128.246.102 : 4.0%
147.32.80.9 : 3.0%
217.163.21.36 : 2.0%
217.163.21.39 : 2.0%


Reservoir size = 1000
193.23.181.44 : 10.4%
174.37.196.55 : 7.3999999999999995%
174.128.246.102 : 7.3%
46.4.36.120 : 7.1%
72.20.15.61 : 6.6000000000000005%
173.236.31.226 : 5.5%
67.19.72.206 : 5.4%
184.154.89.154 : 5.3%
147.32.80.9 : 2.0%
217.163.21.36 : 2.0%


Reservoir size = 5000
193.23.181.44 : 12.34%
174.128.246.102 : 7.08%
174.37.196.55 : 6.84%
67.19.72.206 : 6.32%
173.236.31.226 : 5.94%
184.154.89.154 : 5.82%
46.4.36.120 : 5.76%
72.20.15.61 : 5.2%
147.32.80.9 : 2.6%
212.117.171.138 : 1.8599999999999999%


Reservoir size = 10000
193.23.181.44 : 11.58%
174.128.246.102 : 7.359999999999999%
184.154.89.154 : 6.419999999999999%
174.37.196.55 : 6.370000000000001%
173.236.31.226 : 6.15%
46.4.36.120 : 6.0%
67.19.72.206 : 5.7299999999999995%
72